In [1]:
#Name: Georgios Strouggis
#AM: 5357
#Kaggle Team: AI Slop Squad

#Preprocessing
import gensim.downloader as api
from nltk.corpus import stopwords
import re
import igraph as ig
import random

#Feature Extraction / Document-Term Matrix
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#Dimensional Reduction
from sklearn.decomposition import TruncatedSVD

#Model Learning
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.calibration import CalibratedClassifierCV

#Text Categorization and Evaluation
from sklearn.model_selection import cross_val_score

#Others
import csv

In [2]:
#Preprocessing for Abstracts and Authors .txt files
word2vec = api.load("word2vec-google-news-300") #Loads Word2Vec model
stop_words = set(stopwords.words('english')) #Loads common stop words

first_line_count = 1
first_line = ""

#Opens abstracts.txt and turns each line into a list of words
abstracts = {}
with (open('abstracts.txt', 'r', encoding='utf-8') as ab): 
    for line in ab:
        if first_line_count == 1: 
            first_line = line
            first_line_count = 0
        paper_id, abstract = line.split('|--|') 
        abstract = abstract.lower() 
        abstract = re.sub(r'[^\w\s]', '', abstract) 
        abstract_tokens = abstract.split()
        abstract_tokens = [word for word in abstract_tokens if word not in stop_words]
        abstracts[int(paper_id)] = abstract_tokens

print("Abstract before preprocessing:")
print(first_line) #Shows first line unchanged

print("Same abstract after partial preprocessing")
print(abstracts[0]) #Shows first line after filtering stopwords

#Provides a list of all the words and their frequency in the abstracts overall
common_words = dict()
for i in abstracts:
    for j in range(len(abstracts[i])):
        if common_words.get(abstracts[i][j]) is None:
            common_words[abstracts[i][j]] = 1
        else:
            common_words[abstracts[i][j]] += 1

#Stores list of top 500 most commonly used words from most to least used
top_words = set(sorted(common_words, key=common_words.get, reverse=True)[:500])

#Trims abstracts of top 500 most common words
for paper_id in abstracts:
    abstracts[paper_id] = [word for word in abstracts[paper_id] if word not in top_words]
  
print("\nSame abstract after full preprocessing:")
print(abstracts[0]) #Shows first line after filtering the top 500 most common words

first_line_count = 1
first_line = ""

#Opens authors.txt and turns each line into a list of words
authors = {}
with open('authors.txt', 'r', encoding='utf-8') as au:
    for line in au:
        if first_line_count == 1: 
            first_line = line
            first_line_count = 0
        paper_id, author = line.split('|--|')
        author = author.strip()
        author = author.split(',')
        authors[int(paper_id)] = author

print("\nAbstract's authors before preprocessing:")
print(first_line) #Shows first line unchanged
print("Abstract's authors after preprocessing:")
print(authors[0])

Abstract before preprocessing:
0|--|The development of an automated system for the quality assessment of aerodrome ground lighting (AGL), in accordance with associated standards and recommendations, is presented. The system is composed of an image sensor, placed inside the cockpit of an aircraft to record images of the AGL during a normal descent to an aerodrome. A model-based methodology is used to ascertain the optimum match between a template of the AGL and the actual image data in order to calculate the position and orientation of the camera at the instant the image was acquired. The camera position and orientation data are used along with the pixel grey level for each imaged luminaire, to estimate a value for the luminous intensity of a given luminaire. This can then be compared with the expected brightness for that luminaire to ensure it is operating to the required standards. As such, a metric for the quality of the AGL pattern is determined. Experiments on real image data is pr

In [3]:
#Preprocessing for Edgelist
g = ig.Graph(directed=False) #Creates undirected iGraph

#Opens edgelist.txt and stores nodes from it
nodes = []
with open('edgelist.txt', 'r', encoding='utf-8') as ed:
    for line in ed:
        node1, node2 = line.strip().split(',')
        nodes.append((node1, node2))
 
#1000 randomly selected nodes that will later be used as positive training examples
edge_set = set()
set_count = 1000
used_random_values = []
while set_count > 0:
    new_random = random.randint(0,len(nodes)-1)
    if new_random not in used_random_values:
        edge_set.add((nodes[new_random][0],nodes[new_random][1]))
        used_random_values.append(new_random)
        set_count -= 1
        
g.add_vertices(list(set([v for e in nodes for v in e]))) #Adds the nodes to the graph
g.add_edges(nodes) #Adds edges connecting the nodes to the graph

print("Number of vertices:", g.vcount())
print("Number of edges:", g.ecount())

Number of vertices: 138499
Number of edges: 1091955


In [4]:
#Feature Extraction
#Computes average Word2Vec embedding for a list of tokens
def get_sentence_embedding(tokens):
    vectors = [word2vec[word] for word in tokens if word in word2vec]
    if not vectors:
        return np.zeros((1, word2vec.vector_size))
    return np.mean(vectors, axis=0).reshape(1, -1)

#Computes cosine similarity between the word2vec embeddings of two abstracts
def sentence_embedding_similarity(t1,t2):
    id1 = get_sentence_embedding(abstracts[t1])
    id2 = get_sentence_embedding(abstracts[t2])
    return round(cosine_similarity(id1,id2)[0][0],4)

#Computes jaccard similarity between two lists of authors
def jaccard_similarity(t1, t2):
    union = len(list(set(t1) | set(t2)))
    intersection = len(list(set(t1) & set(t2)))
    return round(intersection/union,2)

#Counts number of common neighbours between two nodes
def common_neighbours(t1,t2):
    neighbors1 = g.neighbors(t1)
    neighbor_names1 = [g.vs[n]['name'] for n in neighbors1]
    neighbors2 = g.neighbors(t2)
    neighbor_names2 = [g.vs[n]['name'] for n in neighbors2]
    return len(list(set(neighbor_names1) & set(neighbor_names2)))

#Computes average degree centrality for two nodes
def degree_centrality(t1,t2):
    id1 = g.degree(str(t1)) / (g.vcount() - 1)
    id2 = g.degree(str(t2)) / (g.vcount() - 1)
    return round(((id1 + id2) / 2),4)

#Computes keyword overlap ratio between two abstracts
def keyword_overlap_ratio(t1,t2):
    id1 = abstracts[t1]
    id2 = abstracts[t2]
    union = len(list(set(id1) | set(id2)))
    intersection = len(list(set(id1) & set(id2)))
    if union != 0:
        return round(intersection/union,4)
    else:
        return 0

#Creates list of pagerank scores for all nodes
pageranks = g.pagerank()
pagerank_dict = {v["name"]: pageranks[v.index] for v in g.vs}

#Computes absolute difference in pagepank scores between two nodes
def pagerank_difference(t1,t2):
    id1 = pagerank_dict.get(str(t1), 0)
    id2 = pagerank_dict.get(str(t2), 0)
    return round(abs(id1 - id2),4)

In [5]:
#Document-Term Matrix
tfidf_vectorizer = TfidfVectorizer(max_features=10000)

set_of_abstracts = {id: ' '.join(tokens) for id, tokens in abstracts.items()} #Set of abstracts fused back into sentences
sorted_set = [set_of_abstracts[id] for id in sorted(set_of_abstracts)] #The above set sorted
tfidf_matrix = tfidf_vectorizer.fit_transform(sorted_set) #Matrix for the frequency of each word in each abstract

lda = LatentDirichletAllocation(n_components=10) #"Finds" the topics of the abstract
topic_features = lda.fit_transform(tfidf_matrix)

#Dimensional Reduction
svd = TruncatedSVD(n_components=300, random_state=42)
tfidf_matrix = svd.fit_transform(tfidf_matrix)

#Sorted list of papers by ids
indexes = {index: id for id, index in enumerate(sorted(set_of_abstracts))}

#Computes cosine similarity between papers using tfidf matrix
def tfidf_similarity(t1, t2):
    id1 = tfidf_matrix[indexes[t1]]
    id2 = tfidf_matrix[indexes[t2]]
    tfidf_sim = cosine_similarity(id1.reshape(1, -1), id2.reshape(1, -1))[0][0]
    return tfidf_sim
  
#Computes cosine similarity between papers using topics
def lda_similarity(t1, t2):
    id1 = topic_features[indexes[t1]]
    id2 = topic_features[indexes[t2]]
    lda_sim = cosine_similarity(id1.reshape(1, -1), id2.reshape(1, -1))[0][0]
    return lda_sim

#Computes all features for two papers
def create_feature_matrix(id1, id2):
    sim1 = sentence_embedding_similarity(id1,id2)
    sim2 = jaccard_similarity(authors[id1], authors[id2])
    cn = common_neighbours(str(id1), str(id2))
    dc = degree_centrality(id1,id2)
    kor = keyword_overlap_ratio(id1, id2)
    pad = pagerank_difference(id1,id2)
    tfidf_sim = tfidf_similarity(id1,id2)
    lda_sim = lda_similarity(id1,id2)

    return [sim1, sim2, cn, dc, kor, pad, tfidf_sim, lda_sim]

In [6]:
#Creates testing data
positive_pairs = list(edge_set)
random.shuffle(positive_pairs)
pairs_num = len(positive_pairs)
negative_pairs = []

#Finds 1000 edges that are known negatives by virtue of not being in the known positives
while pairs_num > 0:
    r1 = random.randint(0,len(abstracts)-1)
    r2 = random.randint(0,len(abstracts)-1)
    if (str(r1), str(r2)) not in edge_set and (str(r2), str(r1)) not in edge_set:
        negative_pairs.append((r1, r2))
        pairs_num -= 1

#Mixes positives and negatives together
labeled_data = [(a, b, 1) for a, b in positive_pairs] + [(a, b, 0) for a, b in negative_pairs]
random.shuffle(labeled_data)

x = []
y = []

#Creates lists of mixed positive and negative examples as well as their labels respectively
for a, b, label in labeled_data:
    features = create_feature_matrix(int(a),int(b))
    x.append(features)
    y.append(label)

In [7]:
#Model Learning, Text Categorization and Evaluation
#Creates list of edges to be tested
test = []
with open('test.txt', 'r', encoding='utf-8') as t:
    for line in t:
        t1, t2 = map(int,line.strip().split(','))
        test.append((t1, t2))

x_test = [create_feature_matrix(id1, id2) for id1, id2 in test]

#Standardizes the training and testing data for better results
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_test_scaled = scaler.transform(x_test)

#Random Forest (uses original data because tree-based algorithms don't need scaling)
rfc = RandomForestClassifier(n_estimators=100, max_depth=10, class_weight='balanced', random_state=42)
scores = cross_val_score(rfc, x, y, cv=5, scoring='f1')
logloss = cross_val_score(rfc, x, y, cv=5, scoring='neg_log_loss')
rfc.fit(x,y)
cal_rfc = CalibratedClassifierCV(rfc, method='isotonic', cv=5) #Calibrates classifier to improve prediction accuracy
cal_rfc.fit(x, y)
probs_rfc = cal_rfc.predict_proba(x_test)[:, 1] #Predict connection between test nodes

print("Cross-validated F1 scores for Random Forest:", scores)
print("Mean F1 score for Random Forest:", np.mean(scores))
print("Cross-validated logloss for Random Forest:", -logloss.mean())

#Logistic Regression
lr = LogisticRegressionCV(cv=5, max_iter=100, class_weight='balanced')
scores = cross_val_score(lr, x_scaled, y, cv=5, scoring='f1')
logloss = cross_val_score(lr, x_scaled, y, cv=5, scoring='neg_log_loss')
lr.fit(x_scaled,y)
cal_lr = CalibratedClassifierCV(lr, method='isotonic', cv=5)
cal_lr.fit(x_scaled, y)
probs_lr = cal_lr.predict_proba(x_test_scaled)[:, 1]

print("\nCross-validated F1 scores for Logistic Regression:", scores)
print("Mean F1 score for Logistic Regression:", np.mean(scores))
print("Cross-validated logloss for Logistic Regression:", -logloss.mean())


#SVC
svc = SVC(probability=True, kernel='linear', class_weight='balanced')
scores = cross_val_score(svc, x_scaled, y, cv=5, scoring='f1')
logloss = cross_val_score(svc, x_scaled, y, cv=5, scoring='neg_log_loss')
svc.fit(x_scaled,y)
cal_svc = CalibratedClassifierCV(svc, method='isotonic', cv=5)
cal_svc.fit(x_scaled, y)
probs_svc = cal_svc.predict_proba(x_test_scaled)[:, 1]

print("\nCross-validated F1 scores for SVC:", scores)
print("Mean F1 score for SVC:", np.mean(scores))
print("Cross-validated logloss for SVC:", -logloss.mean())

#Gradient Boosting (uses original data because tree-based algorithms don't need scaling)
gb = GradientBoostingClassifier(n_estimators=300, learning_rate=0.05, max_depth=3)
scores = cross_val_score(gb, x, y, cv=5, scoring='f1')
logloss = cross_val_score(gb, x, y, cv=5, scoring='neg_log_loss')
gb.fit(x,y)
cal_gb = CalibratedClassifierCV(gb, method='isotonic', cv=5)
cal_gb.fit(x, y)
probs_gb = cal_gb.predict_proba(x_test)[:, 1]

print("\nCross-validated F1 scores for Gradient Boosting:", scores)
print("Mean F1 score for Gradient Boosting:", np.mean(scores))
print("Cross-validated logloss for Gradient Boosting:", -logloss.mean())

#K Neighbours
kn = KNeighborsClassifier(n_neighbors=40)
scores = cross_val_score(kn, x_scaled, y, cv=5, scoring='f1')
logloss = cross_val_score(kn, x_scaled, y, cv=5, scoring='neg_log_loss')
kn.fit(x_scaled,y)
cal_kn = CalibratedClassifierCV(kn, method='isotonic', cv=5)
cal_kn.fit(x_scaled, y)
probs_kn = cal_kn.predict_proba(x_test_scaled)[:, 1]

print("\nCross-validated F1 scores for K Neighbours:", scores)
print("Mean F1 score for K Neighbours:", np.mean(scores))
print("Cross-validated logloss for K Neighbours:", -logloss.mean())

Cross-validated F1 scores for Random Forest: [0.96163683 0.93877551 0.94416244 0.94710327 0.93233083]
Mean F1 score for Random Forest: 0.9448017754047339
Cross-validated logloss for Random Forest: 0.1505906586287944

Cross-validated F1 scores for Logistic Regression: [0.95336788 0.93638677 0.93298969 0.95165394 0.92544987]
Mean F1 score for Logistic Regression: 0.9399696300605613
Cross-validated logloss for Logistic Regression: 0.15996551771434303

Cross-validated F1 scores for SVC: [0.95064935 0.94117647 0.94601542 0.93877551 0.92783505]
Mean F1 score for SVC: 0.9408903614305167
Cross-validated logloss for SVC: 0.16981389142514183

Cross-validated F1 scores for Gradient Boosting: [0.956743   0.93670886 0.94147583 0.93401015 0.93401015]
Mean F1 score for Gradient Boosting: 0.9405895989689123
Cross-validated logloss for Gradient Boosting: 0.15644277582322505

Cross-validated F1 scores for K Neighbours: [0.89071038 0.91906005 0.89304813 0.89528796 0.87978142]
Mean F1 score for K Neighbou

In [8]:
#Write submission file
with open("submission.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["ID", "Label"])
    for id, prob in enumerate(probs_rfc):
    #for id, prob in enumerate(probs_lr):
    #for id, prob in enumerate(probs_svc):
    #for id, prob in enumerate(probs_gb):
    #for id, prob in enumerate(probs_kn):
        writer.writerow([id, prob])

In [11]:
#Additional code implemented to compute feature importance, helped with fine tuning the model to prevent overfitting
feature_names = [
    "sim1", "sim2", "common_neighbors", "degree_centrality", "keyword_overlap", "pagerank_diff", "tfidf_sim", "lda_sim"]
importances = rfc.feature_importances_

print("Feature Importances:")
for name, score in zip(feature_names, importances):
    print(f"{name}: {score:.4f}")

Feature Importances:
sim1: 0.0643
sim2: 0.0093
common_neighbors: 0.4001
degree_centrality: 0.1359
keyword_overlap: 0.1089
pagerank_diff: 0.0340
tfidf_sim: 0.1559
lda_sim: 0.0916
